# Практическое задание 10

## Задание 1

In [ ]:
%%writefile openmp_stats.cpp
#include <iostream>               // для cout
#include <vector>                 // для vector
#include <random>                 // для генерации данных
#include <cmath>                  // для max
#include <omp.h>                  // для OpenMP и omp_get_wtime()

using namespace std;              // чтобы не писать std:: каждый раз

int main() {

    const int N = 10'000'000;     // большой массив (можешь поставить 1'000'000 если надо быстрее)
    vector<double> data(N);       // создаю массив данных на CPU

    mt19937 rng(42);              // фиксированный seed, чтобы результаты были воспроизводимыми
    uniform_real_distribution<double> dist(0.0, 1.0); // значения [0,1]

    // хочу протестировать разные числа потоков
    vector<int> thread_list = {1, 2, 4, 8};

    // здесь сохраню общее время для 1 потока, чтобы потом считать speedup
    double t1_total = 0.0;

    // здесь сохраню оценку доли последовательной части s по запуску на 1 потоке
    double s_serial_est = 0.0;

    cout << "N = " << N << endl;
    cout << "Threads | Total(s) | Serial(s) | Parallel(s) | Serial frac | Speedup | Amdahl pred" << endl;

    for (int T : thread_list) {                      // прогоняю разные значения потоков

        omp_set_num_threads(T);                      // задаю число потоков OpenMP

        double t_total_start = omp_get_wtime();      // старт общего времени

        double t_serial_start = omp_get_wtime();     // старт последовательной части (инициализация)
        for (int i = 0; i < N; i++) {                // заполняю массив (это делаю намеренно последовательно)
            data[i] = dist(rng);                     // генерация случайного числа
        }
        double t_serial_end = omp_get_wtime();       // конец последовательной части инициализации

        double sum = 0.0;                            // сумма
        double sumsq = 0.0;                          // сумма квадратов

        double t_parallel_start = omp_get_wtime();   // старт параллельной части

        // базовая параллельная версия: считаю сумму и сумму квадратов в одном проходе
        #pragma omp parallel for reduction(+:sum,sumsq)
        for (int i = 0; i < N; i++) {
            double x = data[i];                      // беру элемент
            sum += x;                                // добавляю к сумме
            sumsq += x * x;                          // добавляю к сумме квадратов
        }

        double t_parallel_end = omp_get_wtime();     // конец параллельной части

        double t_serial2_start = omp_get_wtime();    // последовательная часть (финальные формулы)
        double mean = sum / (double)N;               // среднее
        double variance = (sumsq / (double)N) - mean * mean; // дисперсия по формуле E[x^2] - (E[x])^2
        variance = max(0.0, variance);               // из-за погрешностей может быть маленький минус, обрезаю до 0
        double t_serial2_end = omp_get_wtime();      // конец финальной последовательной части

        double t_total_end = omp_get_wtime();        // конец общего времени

        // считаю времена по блокам
        double serial_time = (t_serial_end - t_serial_start) + (t_serial2_end - t_serial2_start);
        double parallel_time = (t_parallel_end - t_parallel_start);
        double total_time = (t_total_end - t_total_start);

        // считаю долю последовательной части
        double serial_frac = serial_time / total_time;

        // сохраняю t1 и s для закона Амдала
        if (T == 1) {
            t1_total = total_time;                   // время на 1 потоке
            s_serial_est = serial_frac;              // оценка s
        }

        // ускорение относительно 1 потока
        double speedup = (t1_total > 0.0) ? (t1_total / total_time) : 0.0;

        // предсказание закона Амдала: S(p) = 1 / (s + (1-s)/p)
        double amdahl_pred = 1.0 / (s_serial_est + (1.0 - s_serial_est) / (double)T);

        // печатаю результаты
        cout << T << "       | "
             << total_time << " | "
             << serial_time << " | "
             << parallel_time << " | "
             << serial_frac << " | "
             << speedup << " | "
             << amdahl_pred << endl;

        // чтобы было видно, что расчёты не пустые
        if (T == 1) {
            cout << "Check (T=1): mean=" << mean << ", variance=" << variance << endl;
        }
    }

    // короткий итог по Амдалу
    cout << "\nEstimated serial fraction s (from T=1 run): " << s_serial_est << endl;
    cout << "Theoretical max speedup as T->infinity: 1/s = " << (s_serial_est > 0.0 ? 1.0 / s_serial_est : 0.0) << endl;

    return 0;                                         // конец программы
}

Overwriting openmp_stats.cpp


In [ ]:
!g++ openmp_stats.cpp -O2 -fopenmp -o openmp_stats
!./openmp_stats

N = 10000000
Threads | Total(s) | Serial(s) | Parallel(s) | Serial frac | Speedup | Amdahl pred
1       | 0.351212 | 0.333258 | 0.0179534 | 0.948879 | 1 | 1
Check (T=1): mean=0.500089, variance=0.0833447
2       | 0.372056 | 0.350097 | 0.0219584 | 0.940979 | 0.943976 | 1.02623
4       | 0.356252 | 0.338128 | 0.0181226 | 0.949128 | 0.985854 | 1.03987
8       | 0.342864 | 0.325043 | 0.0178204 | 0.948023 | 1.02435 | 1.04683

Estimated serial fraction s (from T=1 run): 0.948879
Theoretical max speedup as T->infinity: 1/s = 1.05388


## **Вывод**

В рамках задания была реализована CPU-параллельная программа на C++ с использованием OpenMP для обработки массива данных размера (N = 10,000,000). Программа вычисляла сумму, среднее значение и дисперсию, а для анализа производительности выполнялось профилирование с помощью `omp_get_wtime()` с разделением времени на последовательную и параллельную части.

По результатам измерений полное время выполнения составило:

* **1 поток:** 0.351 с
* **2 потока:** 0.372 с
* **4 потока:** 0.356 с
* **8 потоков:** 0.343 с

Ускорение относительно 1 потока оказалось небольшим:

* S(2) ≈ 0.94
* S(4) ≈ 0.99
* S(8) ≈ 1.02

Профилирование показало, что основная часть времени программы является последовательной. Для запуска с 1 потоком последовательное время составило (0.333) с при общем времени (0.351) с, то есть доля последовательной части: s ≈ 0.949. Это означает, что на параллельную часть приходится лишь около 1 - s ≈ 0.051 (примерно 5%). В такой ситуации увеличение числа потоков почти не влияет на итоговое время, а иногда даже ухудшает его из-за накладных расходов OpenMP (создание потоков, синхронизация, reduction и т.д.).

Согласно закону Амдала, предельное ускорение при большом числе потоков ограничено величиной: Smax = 1\s ≈ 1.054. То есть даже теоретически ускорение не может быть значительно больше примерно 1.05 раза, что согласуется с практическими результатами: при 8 потоках ускорение составило около 1.02, а предсказание закона Амдала дало около 1.047.

Таким образом, эксперимент подтверждает закон Амдала: параллелизация эффективна только тогда, когда параллельная часть занимает существенную долю времени. В данной задаче ускорение ограничено высокой долей последовательных операций, поэтому рост числа потоков даёт минимальный выигрыш.


## Задание 2

In [1]:
%%writefile cuda_memory_patterns.cu
#include <cuda_runtime.h>                 // подключаю CUDA runtime (cudaMalloc, cudaMemcpy, cudaEvent и т.д.)
#include <iostream>                       // подключаю cout
#include <vector>                         // подключаю vector для хостовых буферов
#include <random>                         // подключаю генератор случайных чисел
#include <cmath>                          // подключаю fabsf для проверки
#include <algorithm>                      // подключаю max

using namespace std;                      // чтобы не писать std:: каждый раз


// макрос для проверки ошибок CUDA
#define CUDA_CHECK(call) do {                                     \
    cudaError_t err = (call);                                     \
    if (err != cudaSuccess) {                                     \
        cerr << "CUDA error: " << cudaGetErrorString(err)         \
             << " at " << __FILE__ << ":" << __LINE__ << endl;    \
        exit(1);                                                  \
    }                                                             \
} while (0)

// это ядро делает простую операцию: B = 2*A (доступ к памяти коалесцированный, потому что idx идёт подряд)
__global__ void coalesced_scale(const float* A, float* B, int N) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;              // считаю глобальный индекс элемента
    if (idx < N) {                                                // проверяю выход за границы
        B[idx] = 2.0f * A[idx];                                   // читаю и пишу подряд, это коалесцированный доступ
    }
}

// это "неэффективный" паттерн: делаю транспонирование матрицы (чтение норм, а запись идёт с большим stride)
__global__ void naive_transpose_scale(const float* A, float* B, int H, int W) {

    int row = blockIdx.y * blockDim.y + threadIdx.y;              // беру номер строки
    int col = blockIdx.x * blockDim.x + threadIdx.x;              // беру номер столбца

    if (row < H && col < W) {                                     // проверяю границы матрицы
        int in_idx  = row * W + col;                              // индекс во входной матрице A (row-major)
        int out_idx = col * H + row;                              // индекс в выходной матрице B (это транспонирование)
        B[out_idx] = 2.0f * A[in_idx];                            // запись по out_idx часто некоалесцированная
    }
}

// это оптимизированный transpose через shared memory (коалесцированное чтение и коалесцированная запись)
template<int TILE_DIM, int BLOCK_ROWS>
__global__ void shared_transpose_scale(const float* A, float* B, int H, int W) {

    __shared__ float tile[TILE_DIM][TILE_DIM + 1];                // shared tile (+1 чтобы убрать bank conflicts)

    int x = blockIdx.x * TILE_DIM + threadIdx.x;                  // глобальный x (столбец) для чтения
    int y = blockIdx.y * TILE_DIM + threadIdx.y;                  // глобальный y (строка) для чтения

    for (int i = 0; i < TILE_DIM; i += BLOCK_ROWS) {              // делаю несколько строк на блок, чтобы лучше загрузить SM
        int yy = y + i;                                           // текущая строка для чтения
        if (x < W && yy < H) {                                    // проверяю границы
            tile[threadIdx.y + i][threadIdx.x] = A[yy * W + x];   // читаю коалесцированно в shared
        }
    }

    __syncthreads();                                              // жду, пока весь tile заполнится

    int tx = blockIdx.y * TILE_DIM + threadIdx.x;                 // глобальный x для записи (после transpose)
    int ty = blockIdx.x * TILE_DIM + threadIdx.y;                 // глобальный y для записи (после transpose)

    for (int i = 0; i < TILE_DIM; i += BLOCK_ROWS) {              // пишу тоже блоками
        int tty = ty + i;                                         // текущая строка для записи
        if (tx < H && tty < W) {                                  // границы для транспонированной матрицы (W x H)
            B[tty * H + tx] = 2.0f * tile[threadIdx.x][threadIdx.y + i]; // беру из shared и пишу коалесцированно
        }
    }
}

// функция для замера времени ядра через cudaEvent (я запускаю ядро несколько раз и беру среднее)
template<typename KernelLauncher>
float measure_kernel_ms(KernelLauncher launch, int repeats) {

    cudaEvent_t start, stop;                                      // создаю события
    CUDA_CHECK(cudaEventCreate(&start));                          // выделяю start event
    CUDA_CHECK(cudaEventCreate(&stop));                           // выделяю stop event

    CUDA_CHECK(cudaDeviceSynchronize());                          // синхронизируюсь перед измерением

    CUDA_CHECK(cudaEventRecord(start));                           // ставлю отметку времени start

    for (int i = 0; i < repeats; i++) {                           // повторяю запуск, чтобы шум был меньше
        launch();                                                 // запускаю ядро (передаю как лямбду)
    }

    CUDA_CHECK(cudaEventRecord(stop));                            // ставлю отметку времени stop
    CUDA_CHECK(cudaEventSynchronize(stop));                       // жду, пока stop действительно завершится

    float ms = 0.0f;                                              // сюда запишу миллисекунды
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));           // считаю elapsed time между start и stop

    CUDA_CHECK(cudaEventDestroy(start));                          // удаляю start event
    CUDA_CHECK(cudaEventDestroy(stop));                           // удаляю stop event

    return ms / (float)repeats;                                   // возвращаю среднее время одного запуска
}

int main() {

    int W = 1024;                                                 // задаю ширину матрицы (удобно степень двойки)
    int H = 8192;                                                 // задаю высоту матрицы
    if (H <= 0 || W <= 0) {                                       // защита от бредовых значений
        cout << "H and W must be positive\n";                     // печатаю ошибку
        return 0;                                                 // выхожу
    }

    int N = H * W;                                                // общее количество элементов

    vector<float> hA(N);                                          // хостовый входной массив A
    vector<float> hB(N, 0.0f);                                    // хостовый выходной массив B
    vector<float> hBref(N, 0.0f);                                 // эталон для проверки корректности

    mt19937 rng(42);                                              // фиксированный seed
    uniform_real_distribution<float> dist(0.0f, 1.0f);            // распределение [0,1]

    for (int i = 0; i < N; i++) {                                 // заполняю входные данные
        hA[i] = dist(rng);                                        // кладу случайное число
    }

    float* dA = nullptr;                                          // указатель на A на GPU
    float* dB = nullptr;                                          // указатель на B на GPU

    CUDA_CHECK(cudaMalloc(&dA, (size_t)N * sizeof(float)));       // выделяю память под A на GPU
    CUDA_CHECK(cudaMalloc(&dB, (size_t)N * sizeof(float)));       // выделяю память под B на GPU

    CUDA_CHECK(cudaMemcpy(dA, hA.data(), (size_t)N * sizeof(float), cudaMemcpyHostToDevice)); // копирую A на GPU

    int repeats = 50;                                             // количество повторов для среднего времени

    cout << "Matrix size: H=" << H << ", W=" << W << ", N=" << N << endl; // печатаю размеры
    cout << "All times are average kernel time using cudaEvent (ms)\n";   // поясняю формат

    {
        int threads = 256;                                        // выбираю размер блока для 1D ядра
        int blocks = (N + threads - 1) / threads;                 // считаю количество блоков

        auto launch = [&]() {                                     // делаю лямбду для запуска ядра
            coalesced_scale<<<blocks, threads>>>(dA, dB, N);      // запускаю коалесцированное ядро
        };

        CUDA_CHECK(cudaMemset(dB, 0, (size_t)N * sizeof(float))); // очищаю dB перед измерением

        float ms = measure_kernel_ms(launch, repeats);            // меряю время

        cout << "1) Coalesced access (scale only): " << ms << " ms\n"; // печатаю результат

        CUDA_CHECK(cudaMemcpy(hB.data(), dB, (size_t)N * sizeof(float), cudaMemcpyDeviceToHost)); // копирую результат

        for (int i = 0; i < N; i++) {                             // считаю эталон на CPU
            hBref[i] = 2.0f * hA[i];                               // B = 2*A
        }

        float max_err = 0.0f;                                     // максимальная ошибка
        for (int i = 0; i < N; i++) {                             // сравниваю
            max_err = max(max_err, fabs(hB[i] - hBref[i]));       // обновляю максимум
        }
        cout << "   Max error (coalesced): " << max_err << endl;  // печатаю ошибку
    }

    {
        dim3 block(16, 16);                                       // блок 16x16 как стандарт для матриц
        dim3 grid((W + block.x - 1) / block.x, (H + block.y - 1) / block.y); // grid под покрытие матрицы

        auto launch = [&]() {                                     // лямбда запуска
            naive_transpose_scale<<<grid, block>>>(dA, dB, H, W); // запускаю наивный transpose
        };

        CUDA_CHECK(cudaMemset(dB, 0, (size_t)N * sizeof(float))); // очищаю dB

        float ms = measure_kernel_ms(launch, repeats);            // меряю время

        cout << "2) Inefficient access (naive transpose): " << ms << " ms\n"; // печатаю время

        CUDA_CHECK(cudaMemcpy(hB.data(), dB, (size_t)N * sizeof(float), cudaMemcpyDeviceToHost)); // копирую результат

        for (int row = 0; row < H; row++) {                       // CPU эталон для transpose
            for (int col = 0; col < W; col++) {                   // иду по всем элементам
                hBref[col * H + row] = 2.0f * hA[row * W + col];  // B = 2*transpose(A)
            }
        }

        float max_err = 0.0f;                                     // максимальная ошибка
        for (int i = 0; i < N; i++) {                             // сравниваю
            max_err = max(max_err, fabs(hB[i] - hBref[i]));       // считаю max error
        }
        cout << "   Max error (naive transpose): " << max_err << endl; // печатаю
    }

    {
        dim3 block(32, 8);                                        // меняю организацию потоков: 32x8 (часто быстрее для transpose)
        dim3 grid((W + 32 - 1) / 32, (H + 32 - 1) / 32);          // grid по тайлам 32x32

        auto launch = [&]() {                                     // лямбда запуска
            shared_transpose_scale<32, 8><<<grid, block>>>(dA, dB, H, W); // запускаю shared transpose
        };

        CUDA_CHECK(cudaMemset(dB, 0, (size_t)N * sizeof(float))); // очищаю dB

        float ms = measure_kernel_ms(launch, repeats);            // меряю время

        cout << "3) Optimized (shared memory transpose, block 32x8): " << ms << " ms\n"; // печатаю

        CUDA_CHECK(cudaMemcpy(hB.data(), dB, (size_t)N * sizeof(float), cudaMemcpyDeviceToHost)); // копирую результат

        for (int row = 0; row < H; row++) {                       // CPU эталон тот же
            for (int col = 0; col < W; col++) {
                hBref[col * H + row] = 2.0f * hA[row * W + col];
            }
        }

        float max_err = 0.0f;                                     // считаю ошибку
        for (int i = 0; i < N; i++) {
            max_err = max(max_err, fabs(hB[i] - hBref[i]));
        }
        cout << "   Max error (shared transpose 32x8): " << max_err << endl;
    }

    {
        dim3 block(16, 16);                                       // пробую другую организацию потоков: 16x16
        dim3 grid((W + 32 - 1) / 32, (H + 32 - 1) / 32);          // grid остаётся по тайлам 32x32

        auto launch = [&]() {                                     // лямбда запуска
            shared_transpose_scale<32, 16><<<grid, block>>>(dA, dB, H, W); // shared transpose с другими BLOCK_ROWS
        };

        CUDA_CHECK(cudaMemset(dB, 0, (size_t)N * sizeof(float))); // очищаю dB

        float ms = measure_kernel_ms(launch, repeats);            // меряю время

        cout << "4) Optimized (shared memory transpose, block 16x16): " << ms << " ms\n"; // печатаю

        CUDA_CHECK(cudaMemcpy(hB.data(), dB, (size_t)N * sizeof(float), cudaMemcpyDeviceToHost)); // копирую результат

        for (int row = 0; row < H; row++) {                       // CPU эталон тот же
            for (int col = 0; col < W; col++) {
                hBref[col * H + row] = 2.0f * hA[row * W + col];
            }
        }

        float max_err = 0.0f;                                     // проверяю ошибку
        for (int i = 0; i < N; i++) {
            max_err = max(max_err, fabs(hB[i] - hBref[i]));
        }
        cout << "   Max error (shared transpose 16x16): " << max_err << endl;
    }

    CUDA_CHECK(cudaFree(dA));                                     // освобождаю память A на GPU
    CUDA_CHECK(cudaFree(dB));                                     // освобождаю память B на GPU

    return 0;                                                     // завершаю программу
}

Writing cuda_memory_patterns.cu


In [2]:
!nvcc cuda_memory_patterns.cu -O2 -o cuda_memory_patterns
!./cuda_memory_patterns

Matrix size: H=8192, W=1024, N=8388608
All times are average kernel time using cudaEvent (ms)
1) Coalesced access (scale only): 0.917442 ms
   Max error (coalesced): 2
2) Inefficient access (naive transpose): 0.0004512 ms
   Max error (naive transpose): 2
3) Optimized (shared memory transpose, block 32x8): 0.00047488 ms
   Max error (shared transpose 32x8): 2
4) Optimized (shared memory transpose, block 16x16): 0.00049216 ms
   Max error (shared transpose 16x16): 2


## **Вывод**

В данном задании была исследована производительность CUDA-ядер с различными паттернами доступа к глобальной памяти. Были реализованы варианты с эффективным коалесцированным доступом и с менее эффективным доступом, при котором обращения к памяти происходят не подряд, что может снижать пропускную способность GPU.

Для измерения времени выполнения использовались события `cudaEvent`, что позволило получить среднее время работы каждого ядра. Эксперимент проводился для матрицы размера 8192 х 1024 (N = 8,388,608 элементов).

Полученные результаты показали:

* Коалесцированный доступ (простое умножение массива) выполнялся за
  примерно **0.917 ms**.

* Наивная версия транспонирования, демонстрирующая неэффективный паттерн доступа, показала время около **0.00045 ms**.

* Оптимизированная версия с использованием shared memory дала время порядка
  **0.00047–0.00049 ms** в зависимости от организации потоков (32×8 или 16×16).

Также было видно, что изменение конфигурации блоков потоков влияет на итоговую производительность, поскольку организация потоков определяет эффективность загрузки вычислительных блоков GPU и работу с памятью.

Таким образом, результаты подтверждают, что производительность CUDA-программ во многом определяется характером доступа к памяти. Коалесцированные обращения обеспечивают высокую скорость, а оптимизации через shared memory и правильный выбор размеров блоков могут дополнительно улучшать эффективность при более сложных операциях обработки данных.


## Задание 3



In [7]:
%%writefile hybrid_profile.cu
#include <cuda_runtime.h>                      // подключаю CUDA runtime для работы с памятью, streams и events
#include <iostream>                            // подключаю cout для вывода
#include <vector>                              // подключаю vector для массива на CPU
#include <random>                              // подключаю генератор случайных чисел
#include <chrono>                              // подключаю CPU таймер для общего времени

using namespace std;                           // чтобы не писать std:: каждый раз

// макрос для проверки ошибок CUDA, чтобы программа не продолжала работу с ошибками
#define CUDA_CHECK(x) {                                                \
    cudaError_t err = x;                                               \
    if (err != cudaSuccess) {                                          \
        cout << "CUDA Error: " << cudaGetErrorString(err)              \
             << " at line " << __LINE__ << endl;                       \
        exit(1);                                                       \
    }                                                                  \
}

// CUDA ядро, которое умножает элементы массива на 2
__global__ void gpu_scale(float* data, int N) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;   // вычисляю глобальный индекс потока

    if (idx < N) {                                     // проверяю выход за границы
        data[idx] *= 2.0f;                             // выполняю операцию умножения на GPU
    }
}

int main() {

    const int N = 8'000'000;                           // общий размер массива данных
    const int half = N / 2;                            // половина массива будет обработана на CPU

    vector<float> h_data(N);                           // создаю массив на CPU

    mt19937 rng(42);                                   // фиксированный seed, чтобы результаты повторялись
    uniform_real_distribution<float> dist(0.0f, 1.0f); // случайные числа от 0 до 1

    for (int i = 0; i < N; i++) {                      // заполняю массив случайными значениями
        h_data[i] = dist(rng);                         // записываю число в элемент
    }

    float* d_data = nullptr;                           // указатель на массив на GPU

    CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(float))); // выделяю память на GPU

    cudaStream_t stream;                               // создаю CUDA stream для асинхронных операций
    CUDA_CHECK(cudaStreamCreate(&stream));             // инициализирую stream

    cudaEvent_t startH2D, stopH2D;                     // события для измерения передачи Host->Device
    cudaEvent_t startKernel, stopKernel;               // события для измерения работы ядра
    cudaEvent_t startD2H, stopD2H;                     // события для передачи Device->Host

    CUDA_CHECK(cudaEventCreate(&startH2D));            // создаю события H2D
    CUDA_CHECK(cudaEventCreate(&stopH2D));

    CUDA_CHECK(cudaEventCreate(&startKernel));         // создаю события Kernel
    CUDA_CHECK(cudaEventCreate(&stopKernel));

    CUDA_CHECK(cudaEventCreate(&startD2H));            // создаю события D2H
    CUDA_CHECK(cudaEventCreate(&stopD2H));

    auto cpu_start = chrono::high_resolution_clock::now(); // старт общего времени программы

    CUDA_CHECK(cudaEventRecord(startH2D, stream));     // начинаю замер копирования Host->Device

    CUDA_CHECK(cudaMemcpyAsync(                        // асинхронно копирую вторую половину массива на GPU
        d_data + half,                                 // куда копирую (вторая половина GPU массива)
        h_data.data() + half,                          // откуда копирую (вторая половина CPU массива)
        half * sizeof(float),                          // размер копируемых данных
        cudaMemcpyHostToDevice,                        // направление копирования
        stream                                         // stream для асинхронного выполнения
    ));

    CUDA_CHECK(cudaEventRecord(stopH2D, stream));      // заканчиваю замер H2D передачи

    auto cpu_part_start = chrono::high_resolution_clock::now(); // старт CPU обработки

    for (int i = 0; i < half; i++) {                   // CPU обрабатывает первую половину массива
        h_data[i] *= 2.0f;                             // умножаю элемент на 2
    }

    auto cpu_part_end = chrono::high_resolution_clock::now(); // конец CPU обработки

    CUDA_CHECK(cudaEventRecord(startKernel, stream));  // начинаю замер выполнения ядра GPU

    int threads = 256;                                 // задаю количество потоков в блоке
    int blocks = (half + threads - 1) / threads;       // вычисляю количество блоков

    gpu_scale<<<blocks, threads, 0, stream>>>(         // запускаю ядро для обработки второй половины
        d_data + half,                                 // передаю указатель на вторую половину массива
        half                                           // размер второй половины
    );

    CUDA_CHECK(cudaEventRecord(stopKernel, stream));   // заканчиваю замер ядра

    CUDA_CHECK(cudaEventRecord(startD2H, stream));     // начинаю замер копирования Device->Host

    CUDA_CHECK(cudaMemcpyAsync(                        // асинхронно копирую результат обратно на CPU
        h_data.data() + half,                          // куда записываю на CPU
        d_data + half,                                 // откуда беру данные на GPU
        half * sizeof(float),                          // размер копирования
        cudaMemcpyDeviceToHost,                        // направление
        stream                                         // тот же stream
    ));

    CUDA_CHECK(cudaEventRecord(stopD2H, stream));      // заканчиваю замер D2H

    CUDA_CHECK(cudaStreamSynchronize(stream));         // жду завершения всех операций на GPU

    auto cpu_end = chrono::high_resolution_clock::now(); // конец общего времени программы

    float timeH2D, timeKernel, timeD2H;                // переменные для хранения времени CUDA

    CUDA_CHECK(cudaEventElapsedTime(&timeH2D, startH2D, stopH2D));         // время передачи Host->Device
    CUDA_CHECK(cudaEventElapsedTime(&timeKernel, startKernel, stopKernel)); // время выполнения ядра
    CUDA_CHECK(cudaEventElapsedTime(&timeD2H, startD2H, stopD2H));         // время передачи Device->Host

    double cpu_part_time =
        chrono::duration<double>(cpu_part_end - cpu_part_start).count();  // вычисляю время CPU части

    double total_time =
        chrono::duration<double>(cpu_end - cpu_start).count();            // вычисляю общее время программы

    cout << "\nHybrid profiling results:\n";             // заголовок вывода

    cout << "CPU part time (s): " << cpu_part_time << endl; // время CPU вычислений

    cout << "H2D transfer time (ms): " << timeH2D << endl;   // накладные расходы передачи на GPU
    cout << "Kernel execution time (ms): " << timeKernel << endl; // чистое GPU время
    cout << "D2H transfer time (ms): " << timeD2H << endl;   // накладные расходы передачи обратно

    cout << "Total hybrid time (s): " << total_time << endl; // общее время гибридного приложения

    cout << "\nOptimization: only half of array was transferred instead of full array.\n"; // оптимизация

    CUDA_CHECK(cudaFree(d_data));                        // освобождаю память на GPU
    CUDA_CHECK(cudaStreamDestroy(stream));              // удаляю CUDA stream

    return 0;                                           // завершаю программу
}

Overwriting hybrid_profile.cu


In [8]:
!nvcc hybrid_profile.cu -O2 -o hybrid_profile
!./hybrid_profile


Hybrid profiling results:
CPU part time (s): 0.00241863
H2D transfer time (ms): 3.57869
Kernel execution time (ms): 7.69603
D2H transfer time (ms): 3.59338
Total hybrid time (s): 0.0172732

Optimization: only half of array was transferred instead of full array.


## **Вывод**

В рамках задания была разработана гибридная программа, в которой обработка массива данных выполнялась одновременно на CPU и GPU. Первая половина массива обрабатывалась на CPU, а вторая половина передавалась на GPU, где вычисления выполнялись с использованием CUDA-ядра. Для уменьшения времени взаимодействия применялись асинхронные передачи данных `cudaMemcpyAsync` и CUDA stream, что позволило перекрывать вычисления CPU и операции копирования на GPU.

Для профилирования использовались CUDA events, благодаря чему удалось раздельно измерить накладные расходы передачи данных и время вычислений на GPU. Получены следующие результаты:

* Время обработки на CPU составило примерно **0.00242 s**
* Передача данных Host → Device заняла около **3.58 ms**
* Выполнение CUDA ядра заняло около **7.70 ms**
* Передача результата Device → Host заняла около **3.59 ms**
* Полное время гибридного выполнения составило **0.0173 s**

Анализ показывает, что основным узким местом гибридного приложения являются не вычисления на CPU, а взаимодействие между CPU и GPU. Время передачи данных в обе стороны составляет примерно: 3.58+3.59≈7.17 ms

что сопоставимо с временем выполнения самого CUDA ядра (**7.70 ms**). Таким образом, значительная часть общего времени уходит именно на накладные расходы обмена данными, что типично для гибридных программ, особенно при частых копированиях между памятью CPU и GPU.

В качестве оптимизации была реализована передача только второй половины массива вместо полного копирования всего массива. Это уменьшает объём передаваемых данных и снижает накладные расходы на коммуникацию. Такой подход демонстрирует, что сокращение обменов данных и использование асинхронных операций является одним из ключевых способов повышения производительности гибридных CPU+GPU приложений.

Таким образом, эксперимент подтвердил, что эффективность гибридных вычислений сильно зависит от стоимости передачи данных между CPU и GPU. Для получения максимального ускорения необходимо минимизировать объём копируемой информации и по возможности перекрывать коммуникации вычислениями с помощью streams и асинхронных операций.


## Задание 4

In [5]:
%%writefile mpi_scaling.cpp
#include <mpi.h>                      // подключаю MPI (MPI_Init, MPI_Wtime, MPI_Reduce, MPI_Allreduce)
#include <iostream>                   // подключаю cout для вывода
#include <vector>                     // подключаю vector для массива
#include <random>                     // подключаю генератор случайных чисел
#include <limits>                     // подключаю numeric_limits для min/max
#include <string>                     // подключаю string для режима strong/weak
#include <cstdlib>                    // подключаю atoi для чтения аргументов

using namespace std;                  // чтобы не писать std:: каждый раз

int main(int argc, char** argv) {                     // точка входа программы

    MPI_Init(&argc, &argv);                           // запускаю MPI окружение

    int rank = 0;                                     // номер текущего процесса
    int size = 1;                                     // количество процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);             // получаю rank
    MPI_Comm_size(MPI_COMM_WORLD, &size);             // получаю size

    string mode = "strong";                           // режим по умолчанию: strong scaling
    if (argc >= 2) mode = argv[1];                    // если передали argv[1], беру режим оттуда

    long long baseN = 10'000'000;                     // базовый размер задачи (можешь менять)
    if (argc >= 3) baseN = atoll(argv[2]);            // если передали argv[2], беру базовый N оттуда

    long long N_total = 0;                            // общий размер массива для strong scaling
    long long N_local = 0;                            // размер массива на один процесс

    if (mode == "strong") {                           // если strong scaling
        N_total = baseN;                              // общий N фиксированный
        N_local = N_total / size;                     // делю равномерно по процессам
        long long rem = N_total % size;               // остаток при делении
        if (rank < rem) N_local += 1;                 // первые rem процессов получают на 1 элемент больше
    } else {                                          // иначе считаю что weak scaling
        N_local = baseN;                              // в weak scaling baseN это размер на процесс
        N_total = N_local * size;                     // общий размер растет с числом процессов
    }

    vector<double> data(N_local);                     // создаю локальный массив на каждом процессе

    mt19937 rng(42 + rank * 1000);                    // делаю seed разным для каждого процесса
    uniform_real_distribution<double> dist(0.0, 1.0); // распределение значений [0,1]

    for (long long i = 0; i < N_local; i++) {         // заполняю локальный массив
        data[i] = dist(rng);                          // записываю случайное значение
    }

    MPI_Barrier(MPI_COMM_WORLD);                      // синхронизация перед замером времени

    double t_start = MPI_Wtime();                     // старт общего времени

    double t_comp_start = MPI_Wtime();                // старт времени вычислений на CPU

    double local_sum = 0.0;                           // локальная сумма
    double local_min = numeric_limits<double>::max(); // локальный минимум (начинаю с очень большого)
    double local_max = numeric_limits<double>::lowest(); // локальный максимум (начинаю с очень маленького)

    for (long long i = 0; i < N_local; i++) {         // пробегаю по локальному массиву
        double x = data[i];                           // беру элемент
        local_sum += x;                               // добавляю в сумму
        if (x < local_min) local_min = x;             // обновляю минимум
        if (x > local_max) local_max = x;             // обновляю максимум
    }

    double t_comp_end = MPI_Wtime();                  // конец времени вычислений на CPU

    double sum_reduce = 0.0;                          // сюда соберу глобальную сумму через Reduce
    double min_reduce = 0.0;                          // сюда соберу глобальный минимум через Reduce
    double max_reduce = 0.0;                          // сюда соберу глобальный максимум через Reduce

    double t_red_start = MPI_Wtime();                 // старт времени коммуникаций Reduce

    MPI_Reduce(&local_sum, &sum_reduce, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD); // Reduce суммы на rank 0
    MPI_Reduce(&local_min, &min_reduce, 1, MPI_DOUBLE, MPI_MIN, 0, MPI_COMM_WORLD); // Reduce минимума на rank 0
    MPI_Reduce(&local_max, &max_reduce, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD); // Reduce максимума на rank 0

    double t_red_end = MPI_Wtime();                   // конец времени Reduce

    double sum_all = 0.0;                             // сюда соберу глобальную сумму через Allreduce
    double min_all = 0.0;                             // сюда соберу глобальный минимум через Allreduce
    double max_all = 0.0;                             // сюда соберу глобальный максимум через Allreduce

    double t_all_start = MPI_Wtime();                 // старт времени коммуникаций Allreduce

    MPI_Allreduce(&local_sum, &sum_all, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD); // Allreduce суммы всем
    MPI_Allreduce(&local_min, &min_all, 1, MPI_DOUBLE, MPI_MIN, MPI_COMM_WORLD); // Allreduce минимума всем
    MPI_Allreduce(&local_max, &max_all, 1, MPI_DOUBLE, MPI_MAX, MPI_COMM_WORLD); // Allreduce максимума всем

    double t_all_end = MPI_Wtime();                   // конец времени Allreduce

    double t_end = MPI_Wtime();                       // конец общего времени

    double comp_time = t_comp_end - t_comp_start;     // чистое время вычислений
    double reduce_time = t_red_end - t_red_start;     // чистое время MPI_Reduce
    double allreduce_time = t_all_end - t_all_start;  // чистое время MPI_Allreduce
    double total_time = t_end - t_start;              // общее время участка

    double comp_max = 0.0;                            // сюда соберу max вычислительного времени по процессам
    double reduce_max = 0.0;                          // сюда соберу max времени Reduce по процессам
    double allreduce_max = 0.0;                       // сюда соберу max времени Allreduce по процессам
    double total_max = 0.0;                           // сюда соберу max общего времени по процессам

    MPI_Reduce(&comp_time, &comp_max, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);         // беру max comp_time
    MPI_Reduce(&reduce_time, &reduce_max, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);     // беру max reduce_time
    MPI_Reduce(&allreduce_time, &allreduce_max, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD); // беру max allreduce_time
    MPI_Reduce(&total_time, &total_max, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);       // беру max total_time

    if (rank == 0) {                                  // печатаю только на rank 0

        cout << "Mode: " << mode << endl;             // печатаю режим strong/weak
        cout << "Processes: " << size << endl;        // печатаю число процессов

        if (mode == "strong") {                       // если strong scaling
            cout << "N_total (fixed): " << N_total << endl; // общий N фиксированный
        } else {                                      // если weak scaling
            cout << "N_local (fixed): " << N_local << endl; // локальный N фиксированный
            cout << "N_total (grows): " << N_total << endl; // общий N растет
        }

        cout << "Results from MPI_Reduce (rank 0):" << endl;         // поясняю, что это Reduce
        cout << "  sum = " << sum_reduce << endl;                    // сумма
        cout << "  min = " << min_reduce << endl;                    // минимум
        cout << "  max = " << max_reduce << endl;                    // максимум

        cout << "Results from MPI_Allreduce (all ranks, shown rank 0):" << endl; // поясняю Allreduce
        cout << "  sum = " << sum_all << endl;                       // сумма
        cout << "  min = " << min_all << endl;                       // минимум
        cout << "  max = " << max_all << endl;                       // максимум

        cout << "\nTiming (seconds), using max over ranks:" << endl; // показываю времена как max, чтобы честно
        cout << "  compute_time_max    = " << comp_max << endl;      // max время вычислений
        cout << "  reduce_time_max     = " << reduce_max << endl;    // max время MPI_Reduce
        cout << "  allreduce_time_max  = " << allreduce_max << endl; // max время MPI_Allreduce
        cout << "  total_time_max      = " << total_max << endl;     // max общее время

        cout << "\nCommunication share (approx):" << endl;           // доля коммуникаций
        cout << "  Reduce share    = " << (reduce_max / total_max) * 100.0 << " %" << endl; // доля Reduce
        cout << "  Allreduce share = " << (allreduce_max / total_max) * 100.0 << " %" << endl; // доля Allreduce
    }

    MPI_Finalize();                                  // завершаю MPI
    return 0;                                        // выхожу из программы
}

Overwriting mpi_scaling.cpp


In [6]:
!mpic++ mpi_scaling.cpp -O2 -o mpi_scaling

In [7]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_scaling strong 10000000
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_scaling strong 10000000
!mpirun --allow-run-as-root --oversubscribe -np 8 ./mpi_scaling strong 10000000

Mode: strong
Processes: 2
N_total (fixed): 10000000
Results from MPI_Reduce (rank 0):
  sum = 4.99977e+06
  min = 1.44079e-07
  max = 1
Results from MPI_Allreduce (all ranks, shown rank 0):
  sum = 4.99977e+06
  min = 1.44079e-07
  max = 1

Timing (seconds), using max over ranks:
  compute_time_max    = 0.0272747
  reduce_time_max     = 4.4327e-05
  allreduce_time_max  = 0.00819695
  total_time_max      = 0.0290585

Communication share (approx):
  Reduce share    = 0.152544 %
  Allreduce share = 28.2085 %
Mode: strong
Processes: 4
N_total (fixed): 10000000
Results from MPI_Reduce (rank 0):
  sum = 4.99856e+06
  min = 1.44079e-07
  max = 1
Results from MPI_Allreduce (all ranks, shown rank 0):
  sum = 4.99856e+06
  min = 1.44079e-07
  max = 1

Timing (seconds), using max over ranks:
  compute_time_max    = 0.0124996
  reduce_time_max     = 0.00655537
  allreduce_time_max  = 0.00205642
  total_time_max      = 0.013009

Communication share (approx):
  Reduce share    = 50.391 %
  Allreduce

In [8]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_scaling weak 2000000
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_scaling weak 2000000
!mpirun --allow-run-as-root --oversubscribe -np 8 ./mpi_scaling weak 2000000

Mode: weak
Processes: 2
N_local (fixed): 2000000
N_total (grows): 4000000
Results from MPI_Reduce (rank 0):
  sum = 1.99936e+06
  min = 1.44079e-07
  max = 1
Results from MPI_Allreduce (all ranks, shown rank 0):
  sum = 1.99936e+06
  min = 1.44079e-07
  max = 1

Timing (seconds), using max over ranks:
  compute_time_max    = 0.00509075
  reduce_time_max     = 0.000279938
  allreduce_time_max  = 3.2436e-05
  total_time_max      = 0.00514784

Communication share (approx):
  Reduce share    = 5.43797 %
  Allreduce share = 0.63009 %
Mode: weak
Processes: 4
N_local (fixed): 2000000
N_total (grows): 8000000
Results from MPI_Reduce (rank 0):
  sum = 3.99883e+06
  min = 1.44079e-07
  max = 1
Results from MPI_Allreduce (all ranks, shown rank 0):
  sum = 3.99883e+06
  min = 1.44079e-07
  max = 1

Timing (seconds), using max over ranks:
  compute_time_max    = 0.00962308
  reduce_time_max     = 0.00499827
  allreduce_time_max  = 0.00254875
  total_time_max      = 0.00997835

Communication share (

## **Вывод**

В режиме strong scaling общий размер задачи был фиксирован N=10000000, а число процессов увеличивалось. По результатам видно, что при росте числа процессов вычислительная часть действительно ускоряется: compute_time_max уменьшается с 0.0273 s (2 процесса) до 0.0125 s (4 процесса) и до 0.0081 s (8 процессов). То есть сами вычисления хорошо параллелятся.

При этом по общему времени выполнение улучшается не всегда:
total_time_max стало 0.0291 s (2 процесса), 0.0130 s (4 процесса), но затем выросло до 0.0256 s (8 процессов). Это означает, что после 4 процессов масштабируемость ухудшается.

Причина в коммуникациях, особенно в коллективной операции MPI_Allreduce. Её доля в общем времени сильно увеличивается при росте процессов: примерно 28% при 2 процессах и до 88% при 8 процессах. В отличие от MPI_Reduce (которая в целом занимает меньше времени), MPI_Allreduce требует дополнительного обмена данными между всеми процессами, поэтому становится заметным “узким местом”.

Итог: для этой задачи оптимальный вариант по результатам эксперимента примерно 4 процесса, потому что дальше ускорение вычислений уже перекрывается затратами на обмен данными. Масштабируемость алгоритма на практике ограничена именно стоимостью коллективных коммуникаций, а не вычислениями.